In [1]:
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
from pprint import pprint
from sqlalchemy import create_engine
# Google developer API Key
from config import gkey

In [38]:
health_inspection_df = pd.read_csv("/Users/Pam/Downloads/Food_Inspections.csv")
health_inspection_df

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2552835,JERK VALLEY,JERK VALLEY,2830864.0,Restaurant,All,1556 S PULASKI RD,CHICAGO,IL,60623.0,03/14/2022,License,Not Ready,NaN,41.859014,-87.725106,"(-87.72510564861969, 41.85901391332313)"
1,2549819,ROTI,ROTI,1797796.0,Restaurant,Risk 1 (High),10 S RIVERSIDE PLZ,CHICAGO,IL,60606.0,02/15/2022,Non-Inspection,No Entry,NaN,41.881664,-87.638887,"(-87.63888744408949, 41.881664289382925)"
2,2549607,Buyk,Buyk,2831377.0,NaN,Risk 3 (Low),3621 N WESTERN AVE,CHICAGO,IL,60618.0,02/09/2022,License,Not Ready,NaN,41.947389,-87.688130,"(-87.68813004392258, 41.94738858055078)"
3,2544432,LUCY'S,LUCY'S,2818186.0,Restaurant,Risk 1 (High),1043 N CALIFORNIA AVE,CHICAGO,IL,60622.0,11/18/2021,License,Pass,NaN,41.900640,-87.696678,"(-87.69667798280055, 41.90063956353709)"
4,2546652,SUBWAY,SUBWAY,2488938.0,Restaurant,Risk 1 (High),251 E HURON ST,CHICAGO,IL,60611.0,01/19/2022,Canvass,Pass,NaN,41.894903,-87.620744,"(-87.62074372623964, 41.89490330235543)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233640,60245,STAR GYROS,STAR GYROS,50600.0,Restaurant,Risk 1 (High),3400 W ROOSEVELT RD,CHICAGO,IL,60624.0,01/06/2010,Complaint,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.866457,-87.710667,"(-87.71066733173923, 41.866457268921884)"
233641,54216,BLUE SHARKS,BLUE SHARKS,1974637.0,Restaurant,Risk 2 (Medium),3450 W CHICAGO AVE,CHICAGO,IL,60651.0,01/06/2010,Complaint,Pass w/ Conditions,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.895597,-87.713449,"(-87.71344866525071, 41.89559733446603)"
233642,164260,Food 4 Less Midwest #552,FOOD 4 LESS,1596210.0,Grocery Store,Risk 1 (High),7030 S ASHLAND AVE,CHICAGO,IL,60636.0,01/08/2010,Complaint,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.765791,-87.663967,"(-87.66396688645497, 41.76579077043187)"
233643,118307,"GREENLEAF FOOD MART, INC.","GREENLEAF FOOD MART, INC.",1964371.0,Grocery Store,Risk 1 (High),1619 S MICHIGAN AVE,CHICAGO,IL,60616.0,01/12/2010,Complaint,Pass w/ Conditions,"1. SOURCE SOUND CONDITION, NO SPOILAGE, FOODS ...",41.859698,-87.623828,"(-87.62382786533588, 41.859697626140715)"


In [13]:
# Google API
# geocoordinates
target_coordinates = "41.882702, -87.619392" #chicago coordinates
target_search = "hot dogs" #This will return restaurants that offer these types of food
target_radius = 10000
target_type = "restaurant"

In [14]:
# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": gkey,
    "page_token": 60
}

In [15]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [32]:
# convert response to json
places_data = response.json()
# Print the json (pretty printed)
#pprint(json.dumps(places_data, indent=4, sort_keys=True))
# Print the name, address, rating, and business status of the first restaurant that appears
#print(places_data["results"][0]["name"])
#print(places_data["results"][0]["vicinity"])
#print(places_data["results"][0]["rating"])
#print(places_data["results"][0]["business_status"])

results = []
for result in places_data["results"]:
    results.append({"name": result["name"], "vicinity": result["vicinity"], "rating": result["rating"], "business_status": result["business_status"]})
google_results_df = pd.DataFrame(results)
google_results_df


,name,vicinity,rating,business_status
0,Byron's Hot Dog,"1017 W Irving Park Rd, Chicago",4.5,OPERATIONAL
1,Jimmy's Red Hots,"4000 W Grand Ave, Chicago",4.6,OPERATIONAL
2,Jacky's Hot Dogs,"5415 S Pulaski Rd, Chicago",4.7,OPERATIONAL
3,Byron's Hot Dogs,"1701 W Lawrence Ave, Chicago",4.3,OPERATIONAL
4,J's Corner Hot Dogs,"3845 N Elston Ave, Chicago",4.4,OPERATIONAL
5,Lulu's Hot Dogs,"1000 S Leavitt St, Chicago",4.4,OPERATIONAL
6,Duk's Red HOTS,"636 N Ashland Ave, Chicago",4.4,OPERATIONAL
7,George's Hot Dogs,"1876 N Damen Ave, Chicago",4.4,OPERATIONAL
8,Hot Dog Station,"4742 N Kimball Ave, Chicago",4.6,OPERATIONAL
9,Portillo's Hot Dogs,"100 W Ontario St, Chicago",4.4,OPERATIONAL


In [17]:
len(results)

20

In [28]:
subset_health_inspection_df = health_inspection_df.loc[health_inspection_df['Address'].str.contains('3845 N Elston'.upper())]

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
6026,2552936,J'S CORNER HOT DOG,J'S CORNER HOT DOG,81673.0,Restaurant,Risk 2 (Medium),3845 N ELSTON AVE,CHICAGO,IL,60618.0,03/15/2022,Complaint,Fail,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.951247,-87.715366,"(-87.7153661896335, 41.95124734078204)"
36793,2352379,J'S CORNER HOT DOG,J'S CORNER HOT DOG,81673.0,Restaurant,Risk 2 (Medium),3845 N ELSTON AVE,CHICAGO,IL,60618.0,11/26/2019,Canvass,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.951247,-87.715366,"(-87.7153661896335, 41.95124734078204)"
76826,2088429,J'S CORNER HOT DOG,J'S CORNER HOT DOG,81673.0,Restaurant,Risk 2 (Medium),3845 N ELSTON AVE,CHICAGO,IL,60618.0,09/19/2017,Canvass,Pass,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,41.951247,-87.715366,"(-87.7153661896335, 41.95124734078204)"
106066,1937612,J'S CORNER HOT DOG,J'S CORNER HOT DOG,81673.0,Restaurant,Risk 2 (Medium),3845 N ELSTON AVE,CHICAGO,IL,60618.0,06/27/2016,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.951247,-87.715366,"(-87.7153661896335, 41.95124734078204)"
122376,1460211,J'S CORNER HOT DOG,J'S CORNER HOT DOG,81673.0,Restaurant,Risk 2 (Medium),3845 N ELSTON AVE,CHICAGO,IL,60618.0,10/27/2015,Canvass,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.951247,-87.715366,"(-87.7153661896335, 41.95124734078204)"
144564,1496523,J'S CORNER HOT DOG,J'S CORNER HOT DOG,81673.0,Restaurant,Risk 2 (Medium),3845 N ELSTON AVE,CHICAGO,IL,60618.0,09/10/2014,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.951247,-87.715366,"(-87.7153661896335, 41.95124734078204)"
162625,1166398,J'S CORNER HOT DOG,J'S CORNER HOT DOG,81673.0,Restaurant,Risk 2 (Medium),3845 N ELSTON AVE,CHICAGO,IL,60618.0,09/13/2013,Canvass,Pass,NaN,41.951247,-87.715366,"(-87.7153661896335, 41.95124734078204)"
179757,1098725,J'S CORNER HOT DOG,J'S CORNER HOT DOG,81673.0,Restaurant,Risk 2 (Medium),3845 N ELSTON AVE,CHICAGO,IL,60618.0,11/27/2012,Canvass,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.951247,-87.715366,"(-87.7153661896335, 41.95124734078204)"
203672,521742,J'S CORNER HOT DOG,J'S CORNER HOT DOG,81673.0,Restaurant,Risk 2 (Medium),3845 N ELSTON AVE,CHICAGO,IL,60618.0,09/13/2011,Complaint,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.951247,-87.715366,"(-87.7153661896335, 41.95124734078204)"
221301,327288,J'S CORNER HOT DOG,J'S CORNER HOT DOG,81673.0,Restaurant,Risk 2 (Medium),3845 N ELSTON AVE,CHICAGO,IL,60618.0,09/15/2010,Canvass,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.951247,-87.715366,"(-87.7153661896335, 41.95124734078204)"


In [52]:
subset_health_inspection_df['Rating'] = pd.Series([1 for x in range(len(subset_health_inspection_df.index))])
subset_health_inspection_df


/var/folders/qy/r52vv03d17g66nslvgpwlh440000gp/T/ipykernel_3564/153407383.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_health_inspection_df['Rating'] = pd.Series([1 for x in range(len(subset_health_inspection_df.index))])


,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location,Rating
19990,2521749,BUDACKI'S DRIVE IN,BUDACKI'S DRIVE IN,2678018.0,Restaurant,Risk 1 (High),4739 N DAMEN AVE,CHICAGO,IL,60625.0,06/22/2021,Canvass,Pass w/ Conditions,22. PROPER COLD HOLDING TEMPERATURES - Comment...,41.968075,-87.67902,"(-87.67901960462584, 41.96807497181902)",NaN
33286,2369493,BUDACKI'S DRIVE IN,BUDACKI'S DRIVE IN,2678018.0,Restaurant,Risk 1 (High),4739 N DAMEN AVE,CHICAGO,IL,60625.0,04/21/2020,Canvass,Pass w/ Conditions,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.968075,-87.67902,"(-87.67901960462584, 41.96807497181902)",NaN
41805,2312456,BUDACKIS DRIVE-IN HOT DOGS,BUDACKIS DRIVE-IN HOT DOGS,1495075.0,Restaurant,Risk 1 (High),4739 N DAMEN AVE,CHICAGO,IL,60625.0,09/17/2019,Canvass Re-Inspection,Pass,39. CONTAMINATION PREVENTED DURING FOOD PREPAR...,41.968075,-87.67902,"(-87.67901960462584, 41.96807497181902)",NaN
42203,2312207,BUDACKIS DRIVE-IN HOT DOGS,BUDACKIS DRIVE-IN HOT DOGS,1495075.0,Restaurant,Risk 1 (High),4739 N DAMEN AVE,CHICAGO,IL,60625.0,09/12/2019,Canvass,Fail,39. CONTAMINATION PREVENTED DURING FOOD PREPAR...,41.968075,-87.67902,"(-87.67901960462584, 41.96807497181902)",NaN
42715,2312464,BUDACKIS DRIVE-IN HOT DOGS,BUDACKIS DRIVE-IN HOT DOGS,1495075.0,Restaurant,Risk 1 (High),4739 N DAMEN AVE,CHICAGO,IL,60625.0,09/17/2019,Canvass,Out of Business,NaN,41.968075,-87.67902,"(-87.67901960462584, 41.96807497181902)",NaN
44284,2312459,BUDACKI'S DRIVE IN,BUDACKI'S DRIVE IN,2678018.0,Restaurant,Risk 1 (High),4739 N DAMEN AVE,CHICAGO,IL,60625.0,09/17/2019,License,Pass,39. CONTAMINATION PREVENTED DURING FOOD PREPAR...,41.968075,-87.67902,"(-87.67901960462584, 41.96807497181902)",NaN
44782,2312208,BUDACKI'S DRIVE IN,BUDACKI'S DRIVE IN,2678018.0,Restaurant,Risk 1 (High),4739 N DAMEN AVE,CHICAGO,IL,60625.0,09/12/2019,License,Not Ready,NaN,41.968075,-87.67902,"(-87.67901960462584, 41.96807497181902)",NaN
45660,2301189,BUDACKI'S DRIVE IN,BUDACKI'S DRIVE IN,2678018.0,Restaurant,Risk 1 (High),4739 N DAMEN AVE,CHICAGO,IL,60625.0,07/16/2019,License,Not Ready,NaN,41.968075,-87.67902,"(-87.67901960462584, 41.96807497181902)",NaN
47145,2301182,BUDACKIS DRIVE-IN HOT DOGS,BUDACKIS DRIVE-IN HOT DOGS,1495075.0,Restaurant,Risk 1 (High),4739 N DAMEN AVE,CHICAGO,IL,60625.0,07/16/2019,Canvass,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.968075,-87.67902,"(-87.67901960462584, 41.96807497181902)",NaN
78061,2089078,BUDACKIS DRIVE-IN HOT DOGS,BUDACKIS DRIVE-IN HOT DOGS,1495075.0,Restaurant,Risk 1 (High),4739 N DAMEN AVE,CHICAGO,IL,60625.0,09/28/2017,Canvass Re-Inspection,Pass,NaN,41.968075,-87.67902,"(-87.67901960462584, 41.96807497181902)",NaN


In [50]:
for i in range (0, len(google_results_df)):
    address = google_results_df.loc[i, "vicinity"]
    address = address.split(",")[0].upper()
    rating = google_results_df.loc[i, "rating"]
    rating = int(rating)
    
    subset_health_inspection_df = health_inspection_df.loc[health_inspection_df['Address'].str.contains(address)]
        
    subset_health_inspection_df['Rating'] = pd.Series([rating for x in range(len(subset_health_inspection_df.index))])
    print(subset_health_inspection_df)
    

/var/folders/qy/r52vv03d17g66nslvgpwlh440000gp/T/ipykernel_3564/3606758882.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_health_inspection_df['Rating'] = pd.Series([rating for x in range(len(subset_health_inspection_df.index))])
/var/folders/qy/r52vv03d17g66nslvgpwlh440000gp/T/ipykernel_3564/3606758882.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_health_inspection_df['Rating'] = pd.Series([rating for x in range(len(subset_health_inspection_df.index))])
/var/folders/qy/r52vv03d17g6

        Inspection ID         DBA Name         AKA Name  License #  \
7595          2546090  BYRONS KITCHENS  BYRONS KITCHENS     3596.0   
25991         2472230  BYRONS KITCHENS  BYRONS KITCHENS     3596.0   
38380         2357050  BYRONS KITCHENS  BYRONS KITCHENS     3596.0   
41091         2356622  BYRONS KITCHENS  BYRONS KITCHENS     3596.0   
53316         2268257  BYRONS KITCHENS  BYRONS KITCHENS     3596.0   
70750         2145691  BYRONS KITCHENS  BYRONS KITCHENS     3596.0   
74337         2146183  BYRONS KITCHENS  BYRONS KITCHENS     3596.0   
86130         2049582  BYRONS KITCHENS  BYRONS KITCHENS     3596.0   
88541         1996060  BYRONS KITCHENS  BYRONS KITCHENS     3596.0   
90036         2051049  BYRONS KITCHENS  BYRONS KITCHENS     3596.0   
108408        1776224  BYRONS KITCHENS  BYRONS KITCHENS     3596.0   
108868        1767218  BYRONS KITCHENS  BYRONS KITCHENS     3596.0   
124747        1424950  BYRONS KITCHENS  BYRONS KITCHENS     3596.0   
134755        142472

/var/folders/qy/r52vv03d17g66nslvgpwlh440000gp/T/ipykernel_3564/3606758882.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_health_inspection_df['Rating'] = pd.Series([rating for x in range(len(subset_health_inspection_df.index))])
/var/folders/qy/r52vv03d17g66nslvgpwlh440000gp/T/ipykernel_3564/3606758882.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_health_inspection_df['Rating'] = pd.Series([rating for x in range(len(subset_health_inspection_df.index))])
/var/folders/qy/r52vv03d17g6

        Inspection ID            DBA Name            AKA Name  License #  \
6026          2552936  J'S CORNER HOT DOG  J'S CORNER HOT DOG    81673.0   
36793         2352379  J'S CORNER HOT DOG  J'S CORNER HOT DOG    81673.0   
76826         2088429  J'S CORNER HOT DOG  J'S CORNER HOT DOG    81673.0   
106066        1937612  J'S CORNER HOT DOG  J'S CORNER HOT DOG    81673.0   
122376        1460211  J'S CORNER HOT DOG  J'S CORNER HOT DOG    81673.0   
144564        1496523  J'S CORNER HOT DOG  J'S CORNER HOT DOG    81673.0   
162625        1166398  J'S CORNER HOT DOG  J'S CORNER HOT DOG    81673.0   
179757        1098725  J'S CORNER HOT DOG  J'S CORNER HOT DOG    81673.0   
203672         521742  J'S CORNER HOT DOG  J'S CORNER HOT DOG    81673.0   
221301         327288  J'S CORNER HOT DOG  J'S CORNER HOT DOG    81673.0   

       Facility Type             Risk             Address     City State  \
6026      Restaurant  Risk 2 (Medium)  3845 N ELSTON AVE   CHICAGO    IL   
36793     R

/var/folders/qy/r52vv03d17g66nslvgpwlh440000gp/T/ipykernel_3564/3606758882.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_health_inspection_df['Rating'] = pd.Series([rating for x in range(len(subset_health_inspection_df.index))])
/var/folders/qy/r52vv03d17g66nslvgpwlh440000gp/T/ipykernel_3564/3606758882.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_health_inspection_df['Rating'] = pd.Series([rating for x in range(len(subset_health_inspection_df.index))])
/var/folders/qy/r52vv03d17g6

        Inspection ID         DBA Name         AKA Name  License #  \
6453          2549239  HOT DOG STATION  HOT DOG STATION  2153527.0   
21490         2472776  HOT DOG STATION  HOT DOG STATION  2153527.0   
26316         2472462  HOT DOG STATION  HOT DOG STATION  2153527.0   
37628         2356775  HOT DOG STATION  HOT DOG STATION  2153527.0   
79199         2104920  HOT DOG STATION  HOT DOG STATION  2153527.0   
117160        1633054  HOT DOG STATION  HOT DOG STATION  2153527.0   
127308        1566907  HOT DOG STATION  HOT DOG STATION  2153527.0   
140575        1497092  HOT DOG STATION  HOT DOG STATION  2153527.0   
163951        1099254  HOT DOG STATION  HOT DOG STATION  2153527.0   
171289        1229225  TAQUERIA ZAPATA  TAQUERIA ZAPATA  1873357.0   
190206        1215007  HOT DOG STATION  HOT DOG STATION  2153527.0   
191765        1229221  HOT DOG STATION  HOT DOG STATION  2153527.0   
206900         521561  TAQUERIA ZAPATA  TAQUERIA ZAPATA  1873357.0   
211412         52153

/var/folders/qy/r52vv03d17g66nslvgpwlh440000gp/T/ipykernel_3564/3606758882.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_health_inspection_df['Rating'] = pd.Series([rating for x in range(len(subset_health_inspection_df.index))])
/var/folders/qy/r52vv03d17g66nslvgpwlh440000gp/T/ipykernel_3564/3606758882.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_health_inspection_df['Rating'] = pd.Series([rating for x in range(len(subset_health_inspection_df.index))])
/var/folders/qy/r52vv03d17g6

        Inspection ID                 DBA Name                 AKA Name  \
11260         2531843  DONALDS FAMOUS HOT DOGS  DONALDS FAMOUS HOT DOGS   
25302         2402093  DONALDS FAMOUS HOT DOGS  DONALDS FAMOUS HOT DOGS   
30984         2401693  DONALDS FAMOUS HOT DOGS  DONALDS FAMOUS HOT DOGS   
45548         2300960  DONALDS FAMOUS HOT DOGS  DONALDS FAMOUS HOT DOGS   
64395         2184727  DONALDS FAMOUS HOT DOGS  DONALDS FAMOUS HOT DOGS   
64631         2184329  DONALDS FAMOUS HOT DOGS  DONALDS FAMOUS HOT DOGS   
84144         2078420  DONALDS FAMOUS HOT DOGS  DONALDS FAMOUS HOT DOGS   
85874         2071863  DONALDS FAMOUS HOT DOGS  DONALDS FAMOUS HOT DOGS   
106152        1953438  DONALDS FAMOUS HOT DOGS  DONALDS FAMOUS HOT DOGS   
116952        1609556  DONALDS FAMOUS HOT DOGS  DONALDS FAMOUS HOT DOGS   
126771        1560014  DONALDS FAMOUS HOT DOGS  DONALDS FAMOUS HOT DOGS   
139457        1363221  DONALDS FAMOUS HOT DOGS  DONALDS FAMOUS HOT DOGS   
143254        1498180  DO

/var/folders/qy/r52vv03d17g66nslvgpwlh440000gp/T/ipykernel_3564/3606758882.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_health_inspection_df['Rating'] = pd.Series([rating for x in range(len(subset_health_inspection_df.index))])
/var/folders/qy/r52vv03d17g66nslvgpwlh440000gp/T/ipykernel_3564/3606758882.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_health_inspection_df['Rating'] = pd.Series([rating for x in range(len(subset_health_inspection_df.index))])
/var/folders/qy/r52vv03d17g6